In [7]:
# 大规模 Transformer 模型 8 比特量化矩阵乘简介 - 基于 Hugging Face Transformers、Accelerate 以及 bitsandbytes
# blog https://hf-mirror.com/blog/hf-bitsandbytes-integration

In [37]:
import torch
X=torch.Tensor([[2, -1, -1],
	[0,3,2],
	[-1,-1,0]
])

W=torch.Tensor([[-1, 0],
	[0, -2],
    [-1, 2]])
Cx=X.abs().max(axis=1).values  # 找出每列的绝对值的最大值
Cw=W.abs().max(axis=0).values  # 找出每行的绝对值的最大值

X_i8=(X*(127/Cx.view(3,1))).to(torch.int8)
W_i8 =(W*(127/Cw.view(1,2))).to(torch.int8)
Out_i32=X_i8.to(torch.int32)@W_i8.to(torch.int32)
Out_f16=Out_i32*(Cx.view(3,1)/127)*(Cw.view(1,2)/127) # 先对结果进行逐列的norm，再对结果进行逐行的norm

In [31]:
X_i8

tensor([[ 127,  -63,  -63],
        [   0,  127,   84],
        [-127, -127,    0]], dtype=torch.int8)

In [32]:
W_i8

tensor([[-127,    0],
        [   0, -127],
        [-127,  127]], dtype=torch.int8)

In [33]:
Out_i32

tensor([[ -8128,      0],
        [-10668,  -5461],
        [ 16129,  16129]], dtype=torch.int32)

In [43]:
Out_f16  # int8量化的结果

tensor([[-1.0079,  0.0000],
        [-1.9843, -2.0315],
        [ 1.0000,  2.0000]])

In [44]:
Out_origin=X@W # 原始float32的相乘结果，可以看到还是有一定的差距

In [41]:
Out_origin

tensor([[-1.,  0.],
        [-2., -2.],
        [ 1.,  2.]])

In [46]:
(Out_f16-Out_origin).std()

tensor(0.0155)